# Part 8: Medium Knowledge Base

In Part 7, you experienced minimal reasoning effort optimized for speed and cost. In this part, we introduce **medium reasoning effort**, the highest level of effort currently available. This effort adds on a retrieval step when it determines the original results were not optimal.

## Step 1: Load Environment Variables

Run below cell to load the configuration for your Azure resources, choose the **.venv** environment that is created for you.

This time you'll create a knowledge base with balanced reasoning capabilities.

> **⚠️ Troubleshooting**
>
> If code cells get stuck and keep spinning, select **Restart** from the notebook toolbar at the top. If the issue persists after a couple of tries, close VS Code completely and reopen it.

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
if os.getenv('KEYLESS','false').lower() == 'true':
    credential = DefaultAzureCredential()    
else:
    credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "medium-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
if os.getenv('KEYLESS','false').lower() == 'true':
    azure_openai_key = ''
else:
    azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

## Step 2: Create Medium Reasoning Knowledge Base

The code below creates a knowledge base using `KnowledgeRetrievalMediumReasoningEffort`. Unlike minimal reasoning, this configuration includes Azure OpenAI for moderate query decomposition and answer synthesis.

Medium reasoning brings back the Azure OpenAI model to enable query planning and answer synthesis, while using `ANSWER_SYNTHESIS` output mode to return synthesized answers instead of raw chunks. The configuration also includes retrieval instructions that guide source selection with natural language.

This balanced approach provides better reasoning capabilities than minimal effort while maintaining reasonable performance and cost efficiency.

In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalMediumReasoningEffort, KnowledgeRetrievalOutputMode, KnowledgeSourceReference, WebKnowledgeSource

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    api_key=azure_openai_key,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
)

ks = WebKnowledgeSource(
    name="web-knowledge-source",
    description="Knowledge source for Web"
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name="hrdocs-knowledge-source"),
        KnowledgeSourceReference(name="web-knowledge-source"),
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    retrieval_reasoning_effort=KnowledgeRetrievalMediumReasoningEffort(),
    retrieval_instructions="Always search hrdocs first, then web if needed."
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

## Step 3: Query Complex Multi-Step Question

The query below demonstrates medium reasoning capabilities with a complex question that requires multiple steps: finding when Zava launched a handheld product, determining the zodiac sign for that year, and describing the product.

This type of multi-step reasoning showcases how medium effort balances sophisticated query decomposition with performance optimization.

In [ ]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams, WebKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)
web_ks_params = WebKnowledgeSourceParams(
    knowledge_source_name="web-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
hrdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="hrdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            Zava was founded in 1985 by visionary entrepreneurs
            In one particular year, they launched a groundbreaking product that revolutionized the industry that you could hold in your hands
            Name the Chinese Zodiac sign for that year and describe the product that was launched.                         
        """)])
    ],
    knowledge_source_params=[
        web_ks_params, hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

## Step 4: Review References and Citations

Let's check out the references used to construct the answer. Medium reasoning effort provides detailed citations while balancing retrieval thoroughness with efficiency.

In [ ]:
import json

references = json.dumps([ref.as_dict() for ref in result.references], indent=2)
print(references)

## Step 5: Analyze Reasoning Activity

Run the cell below to view the activity log that reveals how medium reasoning effort processed your complex query. You'll see evidence of query decomposition, source selection based on retrieval instructions, and the reasoning steps taken to synthesize the final answer.

In [ ]:
import pandas as pd

activity_types = [{"type": a.type} for a in result.activity]

df = pd.DataFrame(activity_types)

print("Activity Log Steps")
df

In [ ]:
activity_content = json.dumps([a.as_dict() for a in result.activity], indent=2)
print("Activity Details")
print(activity_content)

## Summary

You've now experienced medium reasoning knowledge bases that balance sophisticated reasoning capabilities with performance optimization.

**Key concepts to remember:**
- `KnowledgeRetrievalMediumReasoningEffort` balances reasoning sophistication with performance
- Azure OpenAI model enables moderate query decomposition and answer synthesis
- `ANSWER_SYNTHESIS` output mode provides synthesized answers with proper citations
- `retrieval_instructions` guide source selection (hrdocs first, then web if needed)
- Complex multi-step queries demonstrate medium reasoning's capability to handle sophisticated information needs

### What's Next?

🎉 **Congratulations!** You've completed all 8 parts of the LAB511 Knowledge Bases workshop. You now understand the full spectrum of Azure AI Search Knowledge Bases from basic single-source queries to optimized reasoning strategies.

- Clone and Star our [repo](https://github.com/microsoft/ignite25-LAB511-build-agentic-knowledge-bases-next-level-rag-with-azure-ai-search) to explore further.
- Check out [Azure AI Search docs](https://learn.microsoft.com/en-us/azure/search/) for more information.
- Share your feedback for this lab!

You are now ready to build advanced agentic knowledge bases tailored to your specific needs using Azure AI Search!